# Chapter 11: Training Deep Neural Networks

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

Some problems training a deep DNN (deep neural networks):
- Vanishing or exploding gradients problem. Gradients grow smaller and smaller / bigger and bigger and makes training lower layers very hard.
- Not enough training data, or too costly to label.
- Training may be extremely slow.
- Model with millions of parameters risk overfitting.

## 11.1 The Vanishing/Exploding Gradients Problems

Backpropagation computes and propagates the error gradient of each layer from output to input. 

**Vanishing gradients problem** - Gradients get smaller and smaller as it progresses to lower layers. So Gradient Descent leaves the lower layers' connection weights virtually unchanged and never converges to a good solution.

**Exploding gradients problem** - Similar effect but gradients get bigger and bigger and algorithm diverges.

> In general, deep neural networks suffer from unstable gradients; different layers may learn at widely different speeds.

It was discovered that using logistic sigmoid activation function and initializing weights using a normal distribution ($\mu=0, \sigma = 1$) caused this issue.

- Initializing weights using a normal distribution:
    - The variance (the spread) of the outputs of each layer is much greater than its inputs.
    
    - Going forward in the network, the variance keeps increasing after each layer until the activation function saturates (ends up far right/left) at the top layers.

- Logistic sigmoid activation function:
    - Because the variance keeps increasing, inputs become large (negative or positive, "far left/right"), with outputs of 0 or 1 and derivative extremely close to 0.
    - So backpropagation has no error gradient to propagate to the lower layers.

### 11.1.1 Glorot and He Initialization

For the signal to flow properly, we need the variance of the outputs of each layer to be equal to the variance of its inputs, and we need the gradients to have equal variance before and after flowing through a layer in the reverse direction.

> Microphone Amplifier Analogy: Setting knob too close to 0, voice is inaudible but too close to max, voice is too saturated. For a chain of amplifiers, they all need to be set properly so that voice is loud and clear at the end of the chain.  

> Your voice has to come out of each amplifier at the same amplitude as it came in.

It's impossible to guarantee both (output & gradient variances) unless the layer has an equal number of inputs and neurons (*fan-in*, *fan-out*). Glorot and Bengio proposed a good compromise that the connection weights to be initialized randomly according to *Equation 11-1*, where $\text{fan}_\text{avg} = (\text{fan}_\text{in} + \text{fan}_\text{out})/2$ and is called **Xavier initialization** or **Glorot initialization**. This strategy is used for the logistic activation function.

**LeCun initialization** - Equivalent to Glorot initialization when $\text{fan}_\text{in} = \text{fan}_\text{out} $.

**He initialization** - The initialization strategy for the ReLU activation function (and its variants).

In [ ]:
# Default is Glorot with uniform distribution 
# Change to He initialization
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

# He initialization with uniform distribution based on fan_avg rather than fan_in
# Use VarianceScaling
he_avg_init = keras.initializers.VarianceScaling(scale=2, mode='fan_avg',
                                                 distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

### 11.1.2 Nonsaturating Activation Functions

**Dying ReLUs** - During training, some neurons effectively "die," meaning they stop outputting anything other than 0.

A neuron dies when its weights get tweaked in such a way that the weighted sum of its inputs are negative for all instances in the training set. When this happens, it just keeps outputting 0s, and Gradient Descent does not affect it anymore because the gradient of the ReLU function is 0 when its input is negative.

To solve this problem, use **leaky ReLU** where $\text{LeakyReLU}_\alpha(z) = \text{max}(\alpha z, z) $. The hyperparameter $\alpha$ defines how much the function "leaks": it is the slope of the function for $z<0$ and is typically set to $0.01$.

> Leaky ReLU is just like ReLU, but with a small slope for negative values. A small slope ensures that leaky ReLUs never die.

**Exponential linear unit (ELU)** - Outperforms all the ReLU variants. Looks a lot like the ReLU function with some major differences:
- Takes on nagative values when $z<0$, allowing an average output closer to 0 and alleviating the vanishing gradients problem.
- Hyperparameter $\alpha$ defines the value when z is a large negative number (usually set to 1).
- Has nonzero gradient for $z<0$, avoiding the dead neurons problem.
- If $\alpha = 1$, then the function is smooth everywhere including around $z=0$, speeding up Gradient Descent.

The main drawback of the ELU function is that it is slower to compute than the ReLU function and its variants (due to the exponential function).

**Scaled ELU (SELU)** - A scaled variant of the ELU activation function. The network will *self-normalize* ($\mu =0, \sigma =1$) but under specific conditions:
- Input features must be standarized ($\mu =0, \sigma =1$).
- Every hidden layer's weights must be initialized with LeCun normal initialization.
- Network's architecture must be sequential.

> In general, SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic.

> Because ReLU is the most used activation function, many libraries and hardware accelerators provide ReLU-specific optimizations; if speed is your priority, ReLU might be the best choice.

In [ ]:
# On fashion MNIST as example

# For leaky ReLU
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),                  # FROM TEXTBOOK NOTEBOOK 
    keras.layers.Dense(300, kernel_initializer="he_normal"),     # FROM TEXTBOOK NOTEBOOK 
    keras.layers.LeakyReLU(alpha=0.2),                           # Activation function after each layer
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
])

# For PReLU
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),                  # FROM TEXTBOOK NOTEBOOK 
    keras.layers.Dense(300, kernel_initializer="he_normal"),     # FROM TEXTBOOK NOTEBOOK 
    keras.layers.PReLU(),                                        # Activation function after each layer
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.PReLU()
])

# For SELU
layer = keras.layers.Dense(10, activation="selu",
                           kernel_initializer="lecun_normal")

### 11.1.3 Batch Normalization

Although different initialization and a variant activation function reduces the vanishing/exploding gradients problem, it doesn't guarantee that they won't come back during training.

**Batch Normalization** - Adding an operation in the model just before or after the activation function of each hidden layer. This operation simply zero-centers and normalizes each input, then scales and shifts the result using two new parameter vectors per layer: 1 for scaling, other for shifting.

> If you add a BN layer as the very first layer, you do not need to standardize your training set (eg. using `StandardScaler`).

Come testing a new instance, how to calculate the batch mean/standard deviation?

Estimate the final statistics (overall to use on new instance, instead of the batch values) by using a moving average of the layer's input means and standard deviation.

4 parameters are learned in each batch-normalized layer:
- $\mathbf{\gamma}$, the output scale vector
- $\mathbf{\beta}$, the output offset vector
- $\mathbf{\mu}$, the final input mean vector
- $\mathbf{\sigma}$, the final input standard deviation vector

> $\mathbf{\mu}$ and $\mathbf{\sigma}$ are estimated during training, but only used after training (to replace batch input means and standard deviation).

Batch Normalization acts like a regularizer, reducing the need for other regularization techniques.

There is a runtime penalty: the neural network makes slower predictions due to the extra computations required at each layer. Fortunately, it's often possible to fuse the BN layer with the previous layer, after training, thereby avoiding the runtime penalty.

#### Implementing Batch Normalization with Keras

Just add a `BatchNormalization` layer before or after each hidden layer's activation function.

For example, this model applies BN after every hidden layer and as the first layer in the model (after flattening the input images).

In [3]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

Bn layer adds 4 parameters per input: $\gamma, \beta, \mu, \sigma$. So for the first BN layer, $4 \times 784 = 3,136$ parameters.

$\mu, \sigma$ are moving averages and "non-trainable" so $(3,136 + 1,200 + 400) / 2 = 2,368$ the total number of non-trainable parameters in the model.

In [4]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [5]:
# layers.updates method is deprecated
model.layers[1].updates

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer.py:1402: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


[]

In [ ]:
# To add BN layers before the activation functions
# Remove activation function from hidden layers
# Add them as separate layers after BN layers
# Remove bias term from previous layer, 'use_bias=False'

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

`BatchNormalization` class hyperparameters:
- `momentum`: Uses when updating the exponential moving averages. A good momentum value is close to 1.
- `axis`: Determines which axis should be normalized. Defaults to -1, normalizing the last axis (using the means and standard deviations computed across the other axes).

> Note: BN uses batch statistics during training and the "final" statistics after training (ie. the final values of the moving averages).

### 11.1.4 Gradient Clipping

**Gradient Clipping** - Clip the gradients during backpropagation so that they never exceed some threshold in order to mitigate the exploding gradients problem.

In [7]:
# Set clipvalue or clipnorm for Gradient Clipping
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

This optimizer will clip every component of the gradient vector to a value between -1.0 and 1.0.

If you want to ensure that Gradient Clipping does not change the direction of the gradient vector, you should clip by norm by setting `clipnorm` instead of `clipvalue`.

## 11.2 Reusing Pretrained Layers

### 11.2.1 Transfer Learning with Keras

### 11.2.2 Unsupervised Pretraining

### 11.2.3 Pretraining on an Auxiliary Task

## 11.3 Faster Optimizers

### 11.3.1 Momentum Optimization

### 11.3.2 Nesterov Accelerated Gradient

### 11.3.3 AdaGrad

### 11.3.4 RMSProp

### 11.3.5 Adam and Nadam Optimization

### 11.3.6 Learning Rate Scheduling

## 11.4 Avoiding Overfitting Through Regularization

### 11.4.1 $\ell_1$ and $\ell_2$ Regularization

### 11.4.2 Dropout

### 11.4.3 Monte Carlo (MC) Dropout

### 11.4.4 Max-Norm Regularization

## 11.5 Summary and Practical Guidelines